In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data.dataloader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.parametrizations import orthogonal

from torchvision import datasets, transforms
from torch.autograd import Variable
import time
#import torch.legacy.nn as legacy_nn

In [2]:
from torch.utils.data import Dataset, DataLoader
import joblib

In [3]:
cuda = torch.cuda.is_available()

In [4]:
# class Net(nn.Module):
    
#     def __init__(self,input_size, output_size):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(input_size, output_size)
#         self.activation = getattr(F, activation)
        
#         if activation in ['relu', 'leaky_relu']:
#             torch.nn.init.xavier_uniform_(self.fc1.weight,gain=nn.init.calculate_gain(activation))
#         else:
#             torch.nn.init.xavier_uniform_(self.fc1.weight, gain=1)
        
#     def forward(self, x):
#         x = x.view(x.size(0), -1)
#         x = self.fc1(x)
        
#         return x

In [5]:
class AENet(nn.Module):
    
    def __init__(self,input_size, hidden_size = 1000, activation = 'leaky_relu'):
        super(AENet, self).__init__()
        self.fc1 = orthogonal(nn.Linear(input_size, hidden_size))
        self.activation = getattr(F, activation)
        
        if activation in ['relu', 'leaky_relu']:
            torch.nn.init.xavier_uniform_(self.fc1.weight,gain = nn.init.calculate_gain(activation))
        else:
            torch.nn.init.xavier_uniform_(self.fc1.weight, gain=1)
            
        self.fc2 = nn.Linear(hidden_size, input_size, bias=False)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)

        return x

    def forwardToHidden(self, x): 
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.activation(x)

        return x
    
    def encode(self, x):
        x = x.view(x.size(0), -1)
        x = torch.matmul(x, self.fc2.weight)
        
        return self.activation(x)

In [6]:
class MLPELM(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLPELM, self).__init__()
        self.ae1 = AENet(input_size, hidden_size = 1000)
        self.ae2 = AENet(1000, hidden_size = 500)
        self.lin = nn.Linear(500, 2, bias = False)
        
    def forward(self, x):
        #x = x.view(x.size(0), -1)
        x = self.ae1.encode(x)
        x = self.ae2.encode(x)
        x = self.lin(x)
        
        return x

In [7]:
class AEDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.data_dir = data_dir
        train_x, _ = joblib.load(data_dir)
        self.train_x, self.train_t = torch.Tensor(train_x[:, 2, :]), torch.Tensor(train_x[:, 2, :])
        
    def __len__(self):
        return self.train_x.shape[0]
    
    def __getitem__(self, idx):
        
        X = self.train_x[idx]
        t = self.train_t[idx]

        if self.transform:
            X = self.transform(X)
            
        return X, t

In [8]:
class EQDataset(Dataset):
    def __init__(self, data_dir, n, transform=None):
        self.transform = transform
        self.data_dir = data_dir
        train_x, train_y = joblib.load(data_dir)
        self.len = train_x.shape[0]
        self.train_x, self.train_y = torch.Tensor(train_x.reshape((self.len, 3 * n))), torch.Tensor(train_y)
        
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        
        X = self.train_x[idx]
        y = self.train_y[idx]

        if self.transform:
            X = self.transform(X)
            
        return X, y

In [9]:
class pseudoInverse(object):
    def __init__(self, params, C = 1e-2, L = 100):
        self.params = list(params)
        self.w = self.params[-1]
        self.is_cuda = self.w.is_cuda
        self.C = C
        self.L = L
        
        self.w.data.fill_(0)
        self.dimInput = self.w.data.size()[1]
        self.dimOutput = self.w.data.size()[0]

        self.B = Variable(torch.inverse(self.C * torch.eye(self.dimInput)), requires_grad=False)
        if self.is_cuda:
            self.B = self.B.cuda()
            
    def pseudoSparse(self, H, Y):
        print('Big')
        hth = torch.mm(H.t(), H)
        dimInput = H.size()[1]
        I = Variable(torch.eye(dimInput), requires_grad = False)

        if self.is_cuda:
            I = I.cuda()

        if self.L > 0.0:
            mu = torch.mean(H, dim = 0, keepdim = True)
            S = H  - mu
            S = torch.mm(S.t(), S)
            self.B = Variable(torch.inverse(hth.data + self.C * (I.data + self.L * S.data)),requires_grad=False)
        else:
            self.B = Variable(torch.inverse(hth.data + self.C / I.data), requires_grad=False)

        w = torch.mm(self.B, H.t())
        w = torch.mm(w, Y)
        self.w.data = w.t().data

    def pseudoCompress(self, H, Y):
        print('small')
        hht = torch.mm(H, H.t())
        numSamples = H.size()[0]
        I = Variable(torch.eye(numSamples), requires_grad=False)

        if self.is_cuda:
            I = I.cuda()
        self.B = Variable(torch.inverse(hht.data + self.C / I.data), requires_grad=False)

        w = torch.mm(H.t(), self.B)
        w = torch.mm(w, Y)
        self.w.data = w.t().data
        
    def train_elm(self, H, Y, oneHotVectorize=False):
        if oneHotVectorize:
            targets = targets.view(Y.size(0),-1)
            targets = self.oneHotVectorize(targets = targets)
            
        dimOut = H.size()[1]
        dimInput = H.size()[0]
        
        if  dimInput > dimOut:
            self.pseudoSparse(H, Y)
        else:
            self.pseudoCompress(H, Y)
    
    def oneHotVectorize(self,targets):
        oneHotTarget = torch.zeros(targets.size()[0], self.dimOutput)

        for i in range(targets.size()[0]):
            oneHotTarget[i][int(targets[i].item())] = 1

        if self.is_cuda:
            oneHotTarget=oneHotTarget.cuda()
            
        oneHotTarget = Variable(oneHotTarget, requires_grad=False)

        return oneHotTarget

In [10]:
n = 500
train_data = f'../ISMP/dataset/trainset_v2_{n}_label4.jb'
test_data = f'../ISMP/dataset/testset_v2_{n}_label4.jb'

train_dataset = EQDataset(train_data, n)
test_dataset = EQDataset(test_data, n)

train_loader = DataLoader(dataset = train_dataset, batch_size = 6900, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 1726)

In [11]:
# model.train()
# for data, target in train_loader:
#     if cuda:
#         data, target = data.cuda(), target.cuda()
#     data, target = Variable(data), Variable(target)
#     hiddenOut = model.forwardToHidden(data)
#     optimizer.train(hiddenOut, target)
#     output = model.encode(data)

In [12]:
model = MLPELM(1500, 2)
if cuda:
    model.cuda()
#optimizer = pseudoInverse(params = model.parameters())

In [13]:
# model.train()
# for data, target in train_loader:
#     if cuda:
#         data, target = data.cuda(), target.cuda()
        
#     data, target = Variable(data), Variable(target)
#     optimizer.train_multi(data, target)

In [14]:
for name, param in model.named_parameters():
    print(name,'\t', param.shape)

ae1.fc1.bias 	 torch.Size([1000])
ae1.fc1.parametrizations.weight.original 	 torch.Size([1000, 1500])
ae1.fc2.weight 	 torch.Size([1500, 1000])
ae2.fc1.bias 	 torch.Size([500])
ae2.fc1.parametrizations.weight.original 	 torch.Size([500, 1000])
ae2.fc2.weight 	 torch.Size([1000, 500])
lin.weight 	 torch.Size([2, 500])


In [15]:
X, y = train_dataset[:]

In [16]:
model.train()
data = X
for m in ['ae1', 'ae2']:
    ae = getattr(model, m)
    optimizer = pseudoInverse(params = ae.parameters())
    if cuda:
        data = data.cuda()
    if m != 'lin':
        data = Variable(data)
        hiddenOut = ae.forwardToHidden(data)
        optimizer.train_elm(hiddenOut, data)
        data = ae.encode(data)
    else:
        if cuda:
            target = y.cuda()
        optimizer.train_elm(data, target, oneHotVectorize = True)

Big
Big


In [17]:
X_test, y_test = test_dataset[:]

In [18]:
pred = model.forward(X_test.cuda())

In [19]:
pred = pred.data.max(1)[1]

In [20]:
pred.eq(y_test.cuda().data).cpu().sum()

tensor(878)

In [21]:
pred.sum()

tensor(1179, device='cuda:0')

In [22]:
pred = model.forward(X.cuda())

In [23]:
pred = pred.data.max(1)[1]

In [24]:
pred.eq(y.cuda().data).cpu().sum()

tensor(3315)

In [25]:
pred.sum()

tensor(4573, device='cuda:0')